# Scrape Youtube Comments

In [6]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import json


# Initialize YouTube API client
def get_youtube_api_client(api_version='v3', scopes=('https://www.googleapis.com/auth/youtube.force-ssl',)):
    credentials = service_account.Credentials.from_service_account_file(
        'demoproject002-351320-8605cf1a2df6.json', scopes=scopes)
    youtube = build('youtube', api_version, credentials=credentials)
    return youtube

# Fetch comments and their metadata from a video
def get_video_comments_with_metadata(youtube, video_id, part='snippet,replies', max_results=100):
    comments = []
    results = youtube.commentThreads().list(
        part=part,
        videoId=video_id,
        textFormat='plainText',
        maxResults=max_results
    ).execute()

    while results:
        for item in results['items']:
            comment_data = {}
            snippet = item['snippet']['topLevelComment']['snippet']
            
            # Collect various metadata
            comment_data['comment'] = snippet['textDisplay']
            comment_data['author'] = snippet['authorDisplayName']
            comment_data['author_channel_url'] = snippet['authorChannelUrl']
            comment_data['like_count'] = snippet['likeCount']
            comment_data['published_at'] = snippet['publishedAt']
            comment_data['updated_at'] = snippet['updatedAt']
            
            if 'replies' in item:
                # Collect replies to the comment if available
                comment_data['replies'] = [reply['snippet']['textDisplay'] for reply in item['replies']['comments']]
            
            comments.append(comment_data)
        
        # Check for more comments
        if 'nextPageToken' in results:
            results = youtube.commentThreads().list(
                part=part,
                videoId=video_id,
                pageToken=results['nextPageToken'],
                textFormat='plainText',
                maxResults=max_results
            ).execute()
        else:
            break

    return comments

# Modified part to fetch additional metadata
if __name__ == '__main__':
    youtube = get_youtube_api_client()
    video_id = 'QBTwzYKx-kE'
    comments_with_metadata = get_video_comments_with_metadata(youtube, video_id, part='snippet,replies')

    # Write comments and their metadata to a JSON file
    with open('youtube_comments_with_metadata.json', 'w', encoding='utf-8') as f:
        json.dump(comments_with_metadata, f, ensure_ascii=False, indent=4)
# Videos 
# Who has the WORST Customer Service: QBTwzYKx-kE

In [8]:
# we want to scrape comments from YouTube videos
import selenium
from selenium import webdriver as wb
webD = wb.Chrome('chromedriver.exe')
webD.get('https://www.youtube.com/watch?v=HJ1tofzX6Bk')
# we need to scroll down to load all the comments
import time
for i in range(0,100):
    webD.execute_script('window.scrollBy(0,1000)')
    time.sleep(2)
# we need to scrape the comments
comment = webD.find_elements_by_id('content-text')
comments = []
for i in comment:
    comments.append(i.text)
# we need to save the comments in a csv file
import pandas as pd
df = pd.DataFrame(comments)
df.to_csv('comments.csv')
# we need to clean the comments




AttributeError: 'str' object has no attribute 'capabilities'